In [189]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import sparse
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity


In [190]:
df = pd.read_csv('../../data\games.csv')
df.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [191]:
df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [192]:
#df.date.unique()
df['Release date'] = pd.to_datetime(df['Release date'], format='%b %d, %Y', errors='coerce')
select_cond = df['Peak CCU'] > 0
df = df[select_cond]
# df_recency = df.sort_values(by='Release date')
# df.head()

In [193]:
df.isna().sum()

AppID                             0
Name                              0
Release date                     47
Estimated owners                  0
Peak CCU                          0
Required age                      0
Price                             0
DLC count                         0
About the game                   24
Supported languages               0
Full audio languages              0
Reviews                       16461
Header image                      0
Website                        8047
Support url                    8325
Support email                  3946
Windows                           0
Mac                               0
Linux                             0
Metacritic score                  0
Metacritic url                17946
User score                        0
Positive                          0
Negative                          0
Score rank                    20908
Achievements                      0
Recommendations                   0
Notes                       

In [194]:
df.shape

(20926, 39)

In [195]:
percent = .90
column_percentages = df.count() / len(df)

df = df.dropna(axis=1, thresh=len(df) * percent)
df.shape

(20926, 32)

In [196]:
df.isna().sum()

AppID                            0
Name                             0
Release date                    47
Estimated owners                 0
Peak CCU                         0
Required age                     0
Price                            0
DLC count                        0
About the game                  24
Supported languages              0
Full audio languages             0
Header image                     0
Windows                          0
Mac                              0
Linux                            0
Metacritic score                 0
User score                       0
Positive                         0
Negative                         0
Achievements                     0
Recommendations                  0
Average playtime forever         0
Average playtime two weeks       0
Median playtime forever          0
Median playtime two weeks        0
Developers                       0
Publishers                      65
Categories                     305
Genres              

In [197]:
df.dropna(inplace=True)
df.shape

(17730, 32)

In [198]:
df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
5,1469160,Wartune Reborn,2021-02-26,50000 - 100000,68,0,0.00,0,Feel tired of auto-fight? Feel tired of boring...,['English'],...,0,0,0,7Road,7Road,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip...","Turn-Based Combat,Massively Multiplayer,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
6,1659180,TD Worlds,2022-01-09,0 - 20000,3,0,10.99,1,"TD Worlds is a dynamic, highly strategical gam...","['English', 'Russian', 'Danish']",...,0,0,0,MAKSIM VOLKAU,MAKSIM VOLKAU,"Single-player,Steam Achievements,Steam Cloud","Indie,Strategy","Tower Defense,Rogue-lite,RTS,Replay Value,Perm...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
8,1178150,MazM: Jekyll and Hyde,2020-04-02,0 - 20000,1,0,14.99,0,'MazM: Jekyll and Hyde' is a darkly entertaini...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Growing Seeds,"CFK Co., Ltd.","Single-player,Steam Achievements,Full controll...","Adventure,RPG,Simulation,Strategy","Adventure,Simulation,RPG,Strategy,Singleplayer...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
10,1026420,WARSAW,2019-10-02,20000 - 50000,5,0,23.99,0,Use everything at your disposal to help a team...,"['English', 'French', 'German', 'Polish', 'Rus...",...,0,93,0,Pixelated Milk,"Pixelated Milk,gaming company","Single-player,Steam Achievements,Steam Trading...","Indie,RPG","Tactical RPG,Turn-Based Strategy,Wargame,Histo...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
14,1454010,Diary of Lucie,2020-11-25,0 - 20000,3,0,12.99,0,Diary of Lucie is action-roguelite game made w...,"['English', 'Korean']",...,0,0,0,Parange Project,Parange Project,"Single-player,Partial Controller Support,Steam...","Action,Adventure,Indie,RPG,Strategy,Early Access","Action Roguelike,Action,Rogue-lite,RPGMaker,My...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [199]:
df.set_index('Name', inplace=True)
df.head()

,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,1469160,2021-02-26,50000 - 100000,68,0,0.00,0,Feel tired of auto-fight? Feel tired of boring...,['English'],[],...,0,0,0,7Road,7Road,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip...","Turn-Based Combat,Massively Multiplayer,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
TD Worlds,1659180,2022-01-09,0 - 20000,3,0,10.99,1,"TD Worlds is a dynamic, highly strategical gam...","['English', 'Russian', 'Danish']",[],...,0,0,0,MAKSIM VOLKAU,MAKSIM VOLKAU,"Single-player,Steam Achievements,Steam Cloud","Indie,Strategy","Tower Defense,Rogue-lite,RTS,Replay Value,Perm...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
MazM: Jekyll and Hyde,1178150,2020-04-02,0 - 20000,1,0,14.99,0,'MazM: Jekyll and Hyde' is a darkly entertaini...,"['English', 'French', 'Italian', 'German', 'Sp...",[],...,0,0,0,Growing Seeds,"CFK Co., Ltd.","Single-player,Steam Achievements,Full controll...","Adventure,RPG,Simulation,Strategy","Adventure,Simulation,RPG,Strategy,Singleplayer...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
WARSAW,1026420,2019-10-02,20000 - 50000,5,0,23.99,0,Use everything at your disposal to help a team...,"['English', 'French', 'German', 'Polish', 'Rus...",[],...,0,93,0,Pixelated Milk,"Pixelated Milk,gaming company","Single-player,Steam Achievements,Steam Trading...","Indie,RPG","Tactical RPG,Turn-Based Strategy,Wargame,Histo...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
Diary of Lucie,1454010,2020-11-25,0 - 20000,3,0,12.99,0,Diary of Lucie is action-roguelite game made w...,"['English', 'Korean']",[],...,0,0,0,Parange Project,Parange Project,"Single-player,Partial Controller Support,Steam...","Action,Adventure,Indie,RPG,Strategy,Early Access","Action Roguelike,Action,Rogue-lite,RPGMaker,My...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [200]:
df.columns
#removing what I dont think we will use, for sake of space.

Index(['AppID', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age',
       'Price', 'DLC count', 'About the game', 'Supported languages',
       'Full audio languages', 'Header image', 'Windows', 'Mac', 'Linux',
       'Metacritic score', 'User score', 'Positive', 'Negative',
       'Achievements', 'Recommendations', 'Average playtime forever',
       'Average playtime two weeks', 'Median playtime forever',
       'Median playtime two weeks', 'Developers', 'Publishers', 'Categories',
       'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [201]:
col_to_remove = ['AppID','Peak CCU','About the game','Header image','Recommendations','Average playtime forever',
       'Average playtime two weeks', 'Median playtime forever',
       'Median playtime two weeks','Screenshots','Movies','Tags']

df.drop(col_to_remove,axis=1, inplace=True)
column_object = df.dtypes[df.dtypes == 'object'].keys()
column_object


Index(['Estimated owners', 'Supported languages', 'Full audio languages',
       'Developers', 'Publishers', 'Categories', 'Genres'],
      dtype='object')

In [202]:
multi_label_columns = ['Genres','Supported languages','Full audio languages','Categories']
column_object = column_object.drop(multi_label_columns)
column_object

Index(['Estimated owners', 'Developers', 'Publishers'], dtype='object')

In [203]:
#now do multilabelbinarizer
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df.columns

Index(['Release date', 'Estimated owners', 'Required age', 'Price',
       'DLC count', 'Supported languages', 'Full audio languages', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'User score', 'Positive',
       'Negative', 'Achievements', 'Developers', 'Publishers', 'Categories',
       'Genres'],
      dtype='object')

In [204]:
li = list(df.dtypes[df.dtypes == 'object'].keys())
li

['Estimated owners',
 'Supported languages',
 'Full audio languages',
 'Developers',
 'Publishers',
 'Categories',
 'Genres']

In [205]:
df['Categories'] = df['Categories'].str.split(',')
df['Genres'] = df['Genres'].str.split(',')
df.isna().sum()

Release date            0
Estimated owners        0
Required age            0
Price                   0
DLC count               0
Supported languages     0
Full audio languages    0
Windows                 0
Mac                     0
Linux                   0
Metacritic score        0
User score              0
Positive                0
Negative                0
Achievements            0
Developers              0
Publishers              0
Categories              0
Genres                  0
dtype: int64

In [206]:
df.head()

,Release date,Estimated owners,Required age,Price,DLC count,Supported languages,Full audio languages,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Achievements,Developers,Publishers,Categories,Genres
Name,,,,,,,,,,,,,,,,,,,
Wartune Reborn,2021-02-26,50000 - 100000,0,0.00,0,['English'],[],True,False,False,0,0,87,49,0,7Road,7Road,"[Single-player, Multi-player, MMO, PvP, Online...","[Adventure, Casual, Free to Play, Massively Mu..."
TD Worlds,2022-01-09,0 - 20000,0,10.99,1,"['English', 'Russian', 'Danish']",[],True,False,False,0,0,21,7,62,MAKSIM VOLKAU,MAKSIM VOLKAU,"[Single-player, Steam Achievements, Steam Cloud]","[Indie, Strategy]"
MazM: Jekyll and Hyde,2020-04-02,0 - 20000,0,14.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",[],True,False,False,0,0,76,6,25,Growing Seeds,"CFK Co., Ltd.","[Single-player, Steam Achievements, Full contr...","[Adventure, RPG, Simulation, Strategy]"
WARSAW,2019-10-02,20000 - 50000,0,23.99,0,"['English', 'French', 'German', 'Polish', 'Rus...",[],True,False,False,62,0,589,212,34,Pixelated Milk,"Pixelated Milk,gaming company","[Single-player, Steam Achievements, Steam Trad...","[Indie, RPG]"
Diary of Lucie,2020-11-25,0 - 20000,0,12.99,0,"['English', 'Korean']",[],True,False,False,0,0,100,5,0,Parange Project,Parange Project,"[Single-player, Partial Controller Support, St...","[Action, Adventure, Indie, RPG, Strategy, Earl..."


In [207]:

for col in multi_label_columns:
    df_encoded = pd.DataFrame(mlb.fit_transform(df[col]), columns=mlb.classes_,index=df.index)
    df = pd.concat([df,df_encoded], axis=1)
df.head()


,Release date,Estimated owners,Required age,Price,DLC count,Supported languages,Full audio languages,Windows,Mac,Linux,...,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Controller Support,VR Only,VR Support,VR Supported,Valve Anti-Cheat enabled
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,2021-02-26,50000 - 100000,0,0.00,0,['English'],[],True,False,False,...,0,0,0,0,0,0,0,0,0,0
TD Worlds,2022-01-09,0 - 20000,0,10.99,1,"['English', 'Russian', 'Danish']",[],True,False,False,...,0,0,0,0,0,0,0,0,0,0
MazM: Jekyll and Hyde,2020-04-02,0 - 20000,0,14.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",[],True,False,False,...,0,0,0,0,0,0,0,0,0,0
WARSAW,2019-10-02,20000 - 50000,0,23.99,0,"['English', 'French', 'German', 'Polish', 'Rus...",[],True,False,False,...,0,1,0,0,0,0,0,0,0,0
Diary of Lucie,2020-11-25,0 - 20000,0,12.99,0,"['English', 'Korean']",[],True,False,False,...,0,0,0,0,0,0,0,0,0,0


In [208]:
df.drop(columns=multi_label_columns, inplace=True)
df.shape

(17730, 181)

In [209]:
df.head()

,Release date,Estimated owners,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,...,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Controller Support,VR Only,VR Support,VR Supported,Valve Anti-Cheat enabled
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,2021-02-26,50000 - 100000,0,0.00,0,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
TD Worlds,2022-01-09,0 - 20000,0,10.99,1,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
MazM: Jekyll and Hyde,2020-04-02,0 - 20000,0,14.99,0,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
WARSAW,2019-10-02,20000 - 50000,0,23.99,0,True,False,False,62,0,...,0,1,0,0,0,0,0,0,0,0
Diary of Lucie,2020-11-25,0 - 20000,0,12.99,0,True,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [210]:
one_hot_label = pd.get_dummies(df[column_object])
one_hot_label.head(3)

,Estimated owners_0 - 20000,Estimated owners_100000 - 200000,Estimated owners_1000000 - 2000000,Estimated owners_10000000 - 20000000,Estimated owners_100000000 - 200000000,Estimated owners_20000 - 50000,Estimated owners_200000 - 500000,Estimated owners_2000000 - 5000000,Estimated owners_20000000 - 50000000,Estimated owners_50000 - 100000,...,Publishers_高柳堂,"Publishers_鬼谷工作室,Lightning Games",Publishers_鸽屋谷,Publishers_黑心小作坊,Publishers_黑白牛游戏,Publishers_黒子ラボ,Publishers_龙猫饭,Publishers_유진게임즈,Publishers_토리에트,Publishers_🚀 Raiser Games
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
TD Worlds,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MazM: Jekyll and Hyde,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [211]:
df.drop(column_object,axis=1,inplace=True)
df.head()

,Release date,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,...,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Controller Support,VR Only,VR Support,VR Supported,Valve Anti-Cheat enabled
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,2021-02-26,0,0.00,0,True,False,False,0,0,87,...,0,0,0,0,0,0,0,0,0,0
TD Worlds,2022-01-09,0,10.99,1,True,False,False,0,0,21,...,0,0,0,0,0,0,0,0,0,0
MazM: Jekyll and Hyde,2020-04-02,0,14.99,0,True,False,False,0,0,76,...,0,0,0,0,0,0,0,0,0,0
WARSAW,2019-10-02,0,23.99,0,True,False,False,62,0,589,...,0,1,0,0,0,0,0,0,0,0
Diary of Lucie,2020-11-25,0,12.99,0,True,False,False,0,0,100,...,0,0,0,0,0,0,0,0,0,0


In [212]:
df = pd.concat([df,one_hot_label],axis=1)
df.columns

Index(['Release date', 'Required age', 'Price', 'DLC count', 'Windows', 'Mac',
       'Linux', 'Metacritic score', 'User score', 'Positive',
       ...
       'Publishers_高柳堂', 'Publishers_鬼谷工作室,Lightning Games', 'Publishers_鸽屋谷',
       'Publishers_黑心小作坊', 'Publishers_黑白牛游戏', 'Publishers_黒子ラボ',
       'Publishers_龙猫饭', 'Publishers_유진게임즈', 'Publishers_토리에트',
       'Publishers_🚀 Raiser Games'],
      dtype='object', length=21261)

In [213]:
df_for_model = df
df_for_model.drop('Release date', axis=1,inplace=True)

In [214]:
df.columns

Index(['Required age', 'Price', 'DLC count', 'Windows', 'Mac', 'Linux',
       'Metacritic score', 'User score', 'Positive', 'Negative',
       ...
       'Publishers_高柳堂', 'Publishers_鬼谷工作室,Lightning Games', 'Publishers_鸽屋谷',
       'Publishers_黑心小作坊', 'Publishers_黑白牛游戏', 'Publishers_黒子ラボ',
       'Publishers_龙猫饭', 'Publishers_유진게임즈', 'Publishers_토리에트',
       'Publishers_🚀 Raiser Games'],
      dtype='object', length=21260)

In [215]:
# column_numeric = list(df.dtypes[df.dtypes == 'float64'].keys())
# column_numeric.remove('price')
# column_numeric

In [216]:
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(df[column_numeric])
# i=0
# for column in column_numeric:
#     df[column] = scaled[:,i]
#     i += 1
     
df.head()


,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,...,Publishers_高柳堂,"Publishers_鬼谷工作室,Lightning Games",Publishers_鸽屋谷,Publishers_黑心小作坊,Publishers_黑白牛游戏,Publishers_黒子ラボ,Publishers_龙猫饭,Publishers_유진게임즈,Publishers_토리에트,Publishers_🚀 Raiser Games
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,0,0.00,0,True,False,False,0,0,87,49,...,0,0,0,0,0,0,0,0,0,0
TD Worlds,0,10.99,1,True,False,False,0,0,21,7,...,0,0,0,0,0,0,0,0,0,0
MazM: Jekyll and Hyde,0,14.99,0,True,False,False,0,0,76,6,...,0,0,0,0,0,0,0,0,0,0
WARSAW,0,23.99,0,True,False,False,62,0,589,212,...,0,0,0,0,0,0,0,0,0,0
Diary of Lucie,0,12.99,0,True,False,False,0,0,100,5,...,0,0,0,0,0,0,0,0,0,0


In [217]:
df.memory_usage(deep=True).sum() / 1024**2

370.1796598434448

In [218]:
str_obj = df.dtypes[df.dtypes == 'float64'].keys()
str_obj


Index(['Price'], dtype='object')

In [219]:

# Model initiation
model = NearestNeighbors(metric='euclidean')

# Fit model to the data
model.fit(df)

NearestNeighbors(metric='euclidean')

In [220]:

def getRecommended(gamename: str):
    distances, neighbors = model.kneighbors(df.loc[gamename].values.reshape(1, -1), n_neighbors=5)
    
    similar_games = []
    for index in neighbors[0][1:]:
        similar_games.append(df.index[index])
    
    similar_distance = [f"{round(100 - distance, 2)}%" for distance in distances[0][1:]]
    
    return pd.DataFrame(data={"Game": similar_games, "Similarity": similar_distance})



In [221]:

getRecommended('New World')


g:\Anaconda\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Game,Similarity
0,No Man's Sky,-31609.17%
1,Lost Ark,-42641.11%
2,Sid Meier’s Civilization® VI,-48768.44%
3,VRChat,-51575.96%


In [222]:
cosine_sim = cosine_similarity(df)
cosine_sim_df = pd.DataFrame(cosine_sim, index=df.index)
cosine_sim_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,17720,17721,17722,17723,17724,17725,17726,17727,17728,17729
Name,,,,,,,,,,,,,,,,,,,,,
Wartune Reborn,1.000000,0.326346,0.845293,0.977798,0.886253,0.897021,0.961093,0.968209,0.931795,0.992728,...,0.131830,0.193722,0.669045,0.827692,0.702520,0.271588,0.252387,0.016645,0.822225,0.166945
TD Worlds,0.326346,1.000000,0.615432,0.384888,0.339526,0.341630,0.313027,0.334367,0.328373,0.407357,...,0.905292,0.215342,0.348845,0.352880,0.346669,0.237739,0.970393,0.171038,0.614180,0.978269
MazM: Jekyll and Hyde,0.845293,0.615432,1.000000,0.915538,0.947807,0.938470,0.729663,0.918605,0.931879,0.846168,...,0.418104,0.299226,0.830122,0.920572,0.847829,0.435776,0.565497,0.196045,0.987181,0.472073


In [224]:

# Create function to get the game recommendation
def CosineGameRecommended(gamename:str, recommended_games:int=5):
  print(f'If user like playing Game: \n{gamename}\n5 Game that the user might like to play:')
  # Look up the unique value of the game the user likes in the cosine sim dataframe row
  # The unique value (arr) is returned in an ordered form from small to large 
  arr, ind = np.unique(cosine_sim_df.loc[gamename], return_index=True)
  # Input similar game names from the second-last index to the nth-last index
  similar_games = []
  for index in ind[-(recommended_games+1):-1]:
    similar_games.append(df.index[index])
  # Input the cosine scores of similar games starting from the second-last index to the nth-last index.
  cosine_score = []
  for score in arr[-(recommended_games+1):-1]:
    cosine_score.append(score)
  # Return a dataframe with the most recommended game
  return pd.DataFrame(data = {"Game" : similar_games, "Cosine Similarity" : cosine_score}).sort_values(by='Cosine Similarity')

In [231]:
CosineGameRecommended('New World')

If user like playing Game: 
New World
5 Game that the user might like to play:


,Game,Cosine Similarity
0,Generation Zero®,0.999989
1,Evolve Stage 2,0.999993
2,Propnight,0.999996
3,Digimon Masters Online,0.999999
4,Warface,0.999999
